In [21]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit_aer import Aer, noise as qiskit_aer_noise
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

backend = Aer.get_backend("qasm_simulator")

# ----------------
# Build readout noise
# ----------------
def build_readout_noise(p):
    # Confusion matrix
    prob_matrix = [[1-p, p],
                   [p, 1-p]]
    error = qiskit_aer_noise.ReadoutError(prob_matrix)
    noise_model = qiskit_aer_noise.NoiseModel()
    noise_model.add_readout_error(error, [0])
    return noise_model, prob_matrix

# ----------------
# Prepare circuit
# ----------------
def prepare_circuit(state):
    qc = QuantumCircuit(1, 1)
    if state == "|1⟩":
        qc.x(0)
    elif state == "|+⟩":
        qc.h(0)
    qc.measure(0, 0)
    return qc

# ----------------
# Error mitigation
# ----------------
def mitigate(counts, confusion):
    M = np.array(confusion)
    invM = np.linalg.inv(M)
    raw = np.array([counts.get("0",0), counts.get("1",0)])
    corrected = invM @ raw
    corrected = np.maximum(corrected, 0)
    corrected /= np.sum(corrected)
    return corrected

# ----------------
# Run experiment
# ----------------
def run_lab(state, p):
    clear_output(wait=True)

    qc = prepare_circuit(state)
    noise_model, confusion = build_readout_noise(p)

    job = backend.run(
        qc,
        noise_model=noise_model,
        shots=1024
    )
    result = job.result()

    counts = result.get_counts()
    mitigated = mitigate(counts, confusion)

    raw_probs = np.array([
        counts.get("0",0)/1024,
        counts.get("1",0)/1024
    ])

    labels = ["|0⟩", "|1⟩"]

    plt.figure(figsize=(6,4))
    plt.bar(labels, raw_probs, alpha=0.7, label="Noisy")
    plt.bar(labels, mitigated, alpha=0.7, label="Mitigated")
    plt.legend()
    plt.title("Measurement Error Mitigation")
    plt.ylabel("Probability")
    plt.show()

    display(HTML(f"""
    <b>Input State:</b> {state}<br>
    <b>Readout Noise:</b> p = {p}
    """))

# ----------------
# UI
# ----------------
state_widget = widgets.Dropdown(
    options=["|0⟩", "|1⟩", "|+⟩"],
    description="State:",
    layout=widgets.Layout(width='auto')
)

noise_widget = widgets.FloatSlider(
    value=0.1, min=0, max=0.5, step=0.05,
    description="Readout p:",
    continuous_update=False,
    orientation='horizontal',
    layout=widgets.Layout(width='auto')
)

button = widgets.Button(
    description="Run Experiment",
    button_style='primary',
    tooltip='Click to run the quantum experiment',
    layout=widgets.Layout(width='auto', flex='1 1 auto')
)

output = widgets.Output()

def on_click(b):
    with output:
        run_lab(state_widget.value, noise_widget.value)

button.on_click(on_click)

# Group widgets for improved layout
input_controls = widgets.VBox([
    widgets.HTML("<h4>Input Parameters:</h4>"),
    state_widget,
    noise_widget
], layout=widgets.Layout(border='2px solid lightblue', padding='10px', margin='0 0 10px 0'))

action_button_box = widgets.HBox([
    button
], layout=widgets.Layout(justify_content='center', padding='10px', margin='0 0 10px 0'))

clear_output(wait=True)
display(
    widgets.VBox([
        widgets.HTML("<h3>Quantum Measurement Error Mitigation Lab</h3>"),
        input_controls,
        action_button_box,
        output
    ])
)